In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly

import test
import mylayers
import util

# Used to save temporary variables
import pickle
# import asyncio

In [2]:
import os
filename = os.path.basename(globals()['__vsc_ipynb_file__']).split('.')[0]
filename

'0_NN_softmax'

## Read data

In [3]:
df_train = pd.read_csv("./data/fashion-mnist_train.csv")
df_valid = pd.read_csv("./data/fashion-mnist_test.csv")


In [4]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train.shape

(60000, 785)

### Shuffle the data set

In [6]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)


### Seperate feature column and target columns

In [7]:
X_train = np.array(df_train.drop('label', axis=1), dtype="f")
Y_train = df_train.loc[:, 'label']
X_valid = np.array(df_valid.drop('label', axis=1), dtype="f")
Y_valid = df_valid.loc[:, 'label']


### One hot encoding Y

In [8]:
encoder = util.onehot_encoder(Y_train)
Y_train = encoder.encoding(Y_train)
Y_valid = encoder.encoding(Y_valid)


In [9]:
# X_train = X_train[:1000]
# Y_train = Y_train[:1000]

## Build layers

In [10]:
layers = []
# ---Input Layer---
layers.append(mylayers.InputLayer(X_train))
# ---Hidden Layer---
# initialize FCL with xaiver
fc_layer = mylayers.FullyConnectedLayer(X_train.shape[1], 10)
weights, biases = util.xaiver_init(X_train.shape[1], 10)
fc_layer.setWeights(weights)
fc_layer.setBiases(biases)
layers.append(fc_layer)
layers.append(mylayers.SoftmaxLayer())
# ---Output Layer---
# fc_layer = mylayers.FullyConnectedLayer(20, 10)
# weights, biases = util.xaiver_init(20, 10)
# fc_layer.setWeights(weights)
# fc_layer.setBiases(biases)
# layers.append(fc_layer)
# layers.append(mylayers.SoftmaxLayer())
# ---Objective---
layers.append(mylayers.CrossEntropy())


### layers test

In [11]:
hyperparams = {"epoch": 500, "eta": 10e-8, "batch_size": 5, "rho1": 0.9, "rho2": 0.999}


In [12]:
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,ls_accuracy_train,ls_scores_valid,ls_accuracy_valid = test.adam_learn_with_validation(X_train,Y_train,X_valid,Y_valid,layers,mylayers.CrossEntropy,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


100%|██████████| 500/500 [49:52<00:00,  5.98s/it]


0 await
17.6s
28.42it/s

run with 3 await on fit 
17s 34.2
28.15it/s




run with 5 awit on learn + fit + update kernal
17.8s 33.6
28.07it/s



In [13]:
train_results = {
    'hyperparams' : hyperparams,
    'ls_scores_train':ls_scores_train,
    'Y_hat':Y_hat,
    'ls_accuracy_train':ls_accuracy_train
    ,'ls_scores_valid':ls_scores_valid,
    'ls_accuracy_valid':ls_accuracy_valid
}

In [14]:
# save model
with open(f"models/model_{filename}.pkl", "wb") as f:
    pickle.dump(layers, f)

# save result to pickle
with open(f"models/{filename}.pkl", "wb") as f:
    pickle.dump(train_results, f)

In [15]:
# read result from pickle
with open(f'models/{filename}.pkl', 'rb') as f:
    train_results = pickle.load(f)
    
hyperparams = train_results['hyperparams']
ls_scores_train = train_results['ls_scores_train']
Y_hat = train_results['Y_hat']
ls_accuracy_train = train_results['ls_accuracy_train']
ls_scores_valid = train_results['ls_scores_valid']
ls_accuracy_valid = train_results['ls_accuracy_valid']

In [16]:
# graph of scores
hidden_layers = "softmax"
fig = util.draw_scores_with_validation(
    ls_scores_train, ls_scores_valid, hidden_layers, hyperparams
)
fig.show()


In [17]:
# graph of accuracys
fig = util.draw_accuracys_with_validation(
    ls_accuracy_train, ls_accuracy_valid, hidden_layers, hyperparams
)
fig.show()


### validate accuracy

In [18]:
labeled_Y_hat = encoder.decoding(Y_hat)
labeled_Y_hat


[7,
 8,
 8,
 5,
 9,
 1,
 2,
 3,
 6,
 4,
 9,
 0,
 7,
 1,
 4,
 0,
 6,
 2,
 9,
 1,
 2,
 4,
 8,
 0,
 4,
 9,
 1,
 0,
 3,
 5,
 3,
 6,
 2,
 7,
 3,
 3,
 9,
 9,
 4,
 8,
 2,
 2,
 4,
 2,
 1,
 5,
 4,
 4,
 6,
 5,
 1,
 3,
 2,
 5,
 8,
 9,
 7,
 1,
 1,
 2,
 1,
 8,
 4,
 1,
 4,
 8,
 0,
 9,
 4,
 0,
 5,
 6,
 5,
 6,
 6,
 4,
 0,
 5,
 6,
 1,
 1,
 8,
 9,
 7,
 5,
 2,
 9,
 2,
 0,
 3,
 2,
 4,
 4,
 4,
 9,
 7,
 5,
 8,
 7,
 0,
 6,
 0,
 2,
 7,
 3,
 2,
 1,
 6,
 4,
 4,
 5,
 9,
 6,
 7,
 3,
 1,
 8,
 9,
 1,
 5,
 1,
 3,
 2,
 2,
 7,
 8,
 9,
 5,
 9,
 4,
 5,
 3,
 7,
 1,
 4,
 9,
 2,
 3,
 7,
 3,
 6,
 2,
 9,
 7,
 1,
 1,
 5,
 7,
 8,
 1,
 8,
 5,
 1,
 4,
 2,
 7,
 9,
 0,
 1,
 3,
 9,
 0,
 6,
 1,
 2,
 3,
 5,
 4,
 8,
 3,
 0,
 9,
 3,
 6,
 7,
 5,
 4,
 0,
 8,
 1,
 9,
 9,
 6,
 4,
 9,
 2,
 1,
 4,
 3,
 5,
 5,
 4,
 9,
 1,
 2,
 3,
 1,
 6,
 5,
 2,
 7,
 5,
 1,
 1,
 3,
 7,
 0,
 1,
 0,
 6,
 3,
 8,
 0,
 7,
 8,
 1,
 0,
 9,
 2,
 4,
 7,
 7,
 5,
 9,
 7,
 4,
 2,
 5,
 8,
 3,
 2,
 9,
 2,
 6,
 6,
 3,
 0,
 7,
 9,
 1,
 4,
 2,
 0,
 2,
 7,
 8,
 9,
 5,
 8,
 3,


In [19]:
labeled_Y = encoder.decoding(Y_train)
labeled_Y


[7,
 8,
 8,
 5,
 9,
 1,
 2,
 6,
 6,
 2,
 5,
 0,
 7,
 1,
 6,
 0,
 6,
 2,
 9,
 1,
 2,
 4,
 8,
 0,
 4,
 9,
 1,
 0,
 0,
 5,
 1,
 6,
 2,
 7,
 3,
 4,
 9,
 9,
 4,
 8,
 2,
 6,
 0,
 2,
 1,
 5,
 4,
 4,
 6,
 9,
 1,
 3,
 2,
 5,
 8,
 9,
 5,
 1,
 1,
 2,
 3,
 8,
 3,
 1,
 2,
 8,
 0,
 9,
 4,
 0,
 5,
 6,
 5,
 2,
 6,
 2,
 0,
 5,
 2,
 1,
 1,
 6,
 9,
 7,
 5,
 2,
 9,
 2,
 6,
 3,
 2,
 4,
 4,
 2,
 9,
 7,
 5,
 8,
 7,
 0,
 8,
 0,
 2,
 7,
 3,
 2,
 1,
 2,
 4,
 2,
 5,
 9,
 4,
 7,
 3,
 1,
 8,
 9,
 1,
 5,
 1,
 3,
 2,
 2,
 7,
 8,
 9,
 5,
 9,
 4,
 5,
 3,
 7,
 1,
 6,
 9,
 2,
 3,
 7,
 4,
 6,
 2,
 9,
 7,
 1,
 1,
 5,
 7,
 8,
 1,
 8,
 5,
 1,
 2,
 2,
 7,
 9,
 0,
 1,
 4,
 9,
 0,
 6,
 1,
 2,
 6,
 5,
 4,
 8,
 3,
 0,
 9,
 3,
 6,
 7,
 5,
 4,
 0,
 8,
 1,
 9,
 9,
 6,
 6,
 9,
 2,
 1,
 6,
 3,
 5,
 5,
 4,
 9,
 1,
 4,
 3,
 1,
 2,
 5,
 2,
 8,
 5,
 1,
 1,
 3,
 9,
 0,
 1,
 0,
 6,
 3,
 8,
 0,
 5,
 8,
 1,
 0,
 9,
 2,
 2,
 7,
 5,
 5,
 9,
 7,
 6,
 6,
 5,
 8,
 3,
 6,
 9,
 2,
 6,
 2,
 3,
 0,
 7,
 9,
 1,
 4,
 2,
 6,
 2,
 7,
 8,
 9,
 7,
 8,
 3,


In [20]:
c = 0
for i in range(len(labeled_Y)):
    if labeled_Y[i] == labeled_Y_hat[i]:
        c += 1
print("Final Accuracy:", (c / len(labeled_Y)) * 100, "%")


Final Accuracy: 79.43166666666667 %
